# Introduction

The dataset is evaluated with 6 different regression model and the evaluation is logged in <br>
https://wandb.ai/sidsen/car_resale_value/sweeps?workspace=

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import wandb

#regression models
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor

# Config

In [2]:
wandb.login(key='66caffb142c8f258b1c5bc4ca63bca2a87b9e728')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\siddh/.netrc


True

# Import Preprocessed Data

In [3]:
data = pd.read_csv('autos_preprocessed.csv', header=0, sep=',', encoding='Latin1')

# Label Encoding

In [4]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(data[i])
    tr = mapper[i].transform(data[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    data.loc[:, i+'_labels'] = pd.Series(tr, index=data.index)
    
labeled = data[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


# Score Evaluation

In [5]:
def find_scores(Y_actual, Y_pred, X_train):
    mae = mean_absolute_error(Y_actual, Y_pred)
    mse = mean_squared_error(Y_actual, Y_pred)
    rmse = np.sqrt(mse)
    rmsle = np.log(rmse)
    r2 = r2_score(Y_actual, Y_pred)
    n, k = X_train.shape
    adj_r2_score = 1 - ((1-r2)*(n-1)/(n-k-1))
    
    wandb.log({"mae": mae, "mse": mse, 'rmse':rmse, 'rmsle':rmsle, 'r2':r2, 'adj_r2':adj_r2_score})

# Predictive Modeling

## Bagging Regressor

In [6]:
def bagging_regressor():
    config_defaults = {
                'n_estimators':100,
                'max_samples':0.4,
                'bootstrap':True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = BaggingRegressor(
      n_estimators=config.n_estimators, 
      bootstrap=config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [7]:
bagging_regressor_configs = {
    "name":'BaggingRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {
            "values": [100, 200, 300]
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

sweep_id = wandb.sweep(sweep=bagging_regressor_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=bagging_regressor)

Create sweep with ID: c4irkyi9
Sweep URL: https://wandb.ai/sidsen/car_resale_value/sweeps/c4irkyi9


wandb: Agent Starting Run: ag1y1mjm with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: Currently logged in as: sidsen. Use `wandb login --relogin` to force relogin


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\ensemble\_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85283
mae,1356.80618
mse,10275806.3491
r2,0.85284
rmse,3205.58986


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: fojg3q71 with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85466
mae,1341.88518
mse,10148230.21672
r2,0.85466
rmse,3185.6287


wandb: Agent Starting Run: 7h51k2lv with config:
wandb: 	max_samples: 0.6
wandb: 	n_estimators: 300
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\ensemble\_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85489
mae,1340.75808
mse,10132175.24963
r2,0.85489
rmse,3183.1078


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Bagging Regressor Results

<img src="./Bagging Regressor.png" alt="Bagging Regressor Results"/>

## RandomForest Regressor

In [8]:
def random_forest_regressor():
    config_defaults = {
                'n_estimators':100,
                'max_samples':0.4,
                'criterion':'squared_error',
                'bootstrap': True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = RandomForestRegressor(
      n_estimators=config.n_estimators,
      criterion = config.criterion,
      bootstrap=config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [9]:
random_forest_configs = {
    "name":'RandomForestRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {
            "values": [100, 200, 300]
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

sweep_id = wandb.sweep(sweep=random_forest_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=random_forest_regressor)

Create sweep with ID: kxpnpy8e
Sweep URL: https://wandb.ai/sidsen/car_resale_value/sweeps/kxpnpy8e


wandb: Agent Starting Run: nclwou78 with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85234
mae,1356.56346
mse,10309778.53472
r2,0.85235
rmse,3210.88439


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 8gvs3slm with config:
wandb: 	max_samples: 0.4
wandb: 	n_estimators: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85471
mae,1342.53083
mse,10144714.37081
r2,0.85471
rmse,3185.07682


wandb: Agent Starting Run: r9imaout with config:
wandb: 	max_samples: 0.6
wandb: 	n_estimators: 300
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85492
mae,1341.04
mse,10129806.61204
r2,0.85493
rmse,3182.73571


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Random Forest Regressor Results

<img src="./Random Forest Regressor.png" alt="Random Forest Regressor Results"/>

## HistGradient Boosting Regressor

In [10]:
def hist_gradient_boost_regressor():
    config_defaults = {
                'loss':'squared_error',
                'learning_rate': 0.1,
                'max_iter':100,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = HistGradientBoostingRegressor(
      loss=config.loss,
      learning_rate = config.learning_rate,
      max_iter=config.max_iter,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [11]:
hist_gradient_boost_configs = {
    "name":'HistGradientBoostingRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "loss": {
            "values": ['squared_error', 'absolute_error']
        },
        "learning_rate": {
            "values": [0.01, 0.03, 0.05, 0.07]
        },
        "max_iter": {
            "values": [100,200,300]
        },
        "random_state": {
            "values": [42]
        }
    }
}

sweep_id = wandb.sweep(sweep=hist_gradient_boost_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=hist_gradient_boost_regressor)

Create sweep with ID: 5xyixvtv
Sweep URL: https://wandb.ai/sidsen/car_resale_value/sweeps/5xyixvtv


wandb: Agent Starting Run: bew5wy5l with config:
wandb: 	learning_rate: 0.01
wandb: 	loss: squared_error
wandb: 	max_iter: 100
wandb: 	random_state: 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.63695
mae,2858.8648
mse,25348454.01712
r2,0.63698
rmse,5034.72482


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: zklph72b with config:
wandb: 	learning_rate: 0.01
wandb: 	loss: squared_error
wandb: 	max_iter: 200
wandb: 	random_state: 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.82344
mae,1380.87201
mse,12327680.78524
r2,0.82345
rmse,3511.07972


wandb: Agent Starting Run: vd9bl9s6 with config:
wandb: 	learning_rate: 0.07
wandb: 	loss: absolute_error
wandb: 	max_iter: 300
wandb: 	random_state: 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.83392
mae,1344.36996
mse,11596000.98051
r2,0.83393
rmse,3405.29015


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Bagging Regressor Results

<img src="./HOG Boosting Regressor.png" alt="HistGradient Boosting Regressor Results"/>

## Extra Trees Regressor

In [12]:
def extra_tree_regressor():
    config_defaults = {
                'criterion':'squared_error',
                'max_samples':0.4,
                'bootstrap': True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = ExtraTreesRegressor(
      criterion=config.criterion,
      bootstrap = config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [13]:
extra_tree_configs = {
    "name":'ExtraTreesRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "criterion": {
            "values": ['squared_error', 'absolute_error']
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

sweep_id = wandb.sweep(sweep=extra_tree_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=extra_tree_regressor)

Create sweep with ID: g22odoxf
Sweep URL: https://wandb.ai/sidsen/car_resale_value/sweeps/g22odoxf


wandb: Agent Starting Run: 1aw9rttg with config:
wandb: 	criterion: squared_error
wandb: 	max_samples: 0.4
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.8466
mae,1398.26502
mse,10710855.11088
r2,0.84661
rmse,3272.74428


wandb: Agent Starting Run: yt72vcjc with config:
wandb: 	criterion: squared_error
wandb: 	max_samples: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.84886
mae,1377.92734
mse,10552801.52312
r2,0.84887
rmse,3248.50758


wandb: Agent Starting Run: bf2al1gk with config:
wandb: 	criterion: absolute_error
wandb: 	max_samples: 0.6
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85064
mae,1372.7237
mse,10428673.21626
r2,0.85065
rmse,3229.34563


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Extra Tree Regressor Results

<img src="./Extra Tree Regressor.png" alt="Extra Tree Regressor Results"/>

## XGB Regressor

In [14]:
def XGB_regressor():
    config_defaults = {
                'learning_rate':0.1,
                'n_estimators': 500,
                'booster':'gbtree',
                'eta':0.01,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = XGBRegressor(
      learning_rate=config.learning_rate,
      n_estimators = config.n_estimators,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [15]:
extra_tree_configs = {
    "name":'XGBRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.01, 0.03, 0.05, 0.07]
        },
        "n_estimators": {
            "values": [100,200,300]
        },
        "booster": {
            "values": ['gbtree','gblinear']
        },
        "eta": {
            "values": [0.01, 0.03, 0.05, 0.07]
        }
    }
}

sweep_id = wandb.sweep(sweep=extra_tree_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=XGB_regressor)

Create sweep with ID: gu743nfg
Sweep URL: https://wandb.ai/sidsen/car_resale_value/sweeps/gu743nfg


wandb: Agent Starting Run: 8ed91nuq with config:
wandb: 	booster: gbtree
wandb: 	eta: 0.01
wandb: 	learning_rate: 0.01
wandb: 	n_estimators: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.57719
mae,2743.30737
mse,29521641.54611
r2,0.57721
rmse,5433.38215


wandb: Agent Starting Run: i01ha7ps with config:
wandb: 	booster: gbtree
wandb: 	eta: 0.01
wandb: 	learning_rate: 0.01
wandb: 	n_estimators: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.7696
mae,1862.07031
mse,16086655.58174
r2,0.76962
rmse,4010.81732


...

wandb: Agent Starting Run: qjaq2tcq with config:
wandb: 	booster: gblinear
wandb: 	eta: 0.07
wandb: 	learning_rate: 0.07
wandb: 	n_estimators: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.85838
mae,1383.41393
mse,9888414.27207
r2,0.85838
rmse,3144.58491


wandb: Agent Starting Run: pdlgw0zz with config:
wandb: 	booster: gblinear
wandb: 	eta: 0.07
wandb: 	learning_rate: 0.07
wandb: 	n_estimators: 300
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86288
mae,1351.30183
mse,9573756.55093
r2,0.86289
rmse,3094.14876


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## XGB Regressor Results

<img src="./XGB Regressor.png" alt="XGB Regressor Results"/>

## LGBM Regressor

In [16]:
def LGBM_regressor():
    config_defaults = {
                'objective':'root_mean_squared_error',
                'reg_sqrt': True,
                'metric':'rmse',
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = LGBMRegressor(
      learning_rate=config.learning_rate,
      n_estimators = config.n_estimators,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [17]:
lgbm_configs = {
    "name":'LGBMRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.01, 0.03, 0.05, 0.07]
        },
        "objective": {
            "values": ['root_mean_squared_error']
        },
        "boosting_type": {
            "values": ['gbdt','dart','goss','rf']
        },
        "reg_sqrt": {
            "values": [True]
        },
        "metric": {
            "values": ['rmse']
        },
        "n_estimators": {
            "values": [100,200,300]
        },
        "random_state": {
            "values": [42]
        }
    }
}

sweep_id = wandb.sweep(sweep=lgbm_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=LGBM_regressor)

Create sweep with ID: jys7m97k
Sweep URL: https://wandb.ai/sidsen/car_resale_value/sweeps/jys7m97k


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: dwddud6h with config:
wandb: 	boosting_type: gbdt
wandb: 	learning_rate: 0.01
wandb: 	metric: rmse
wandb: 	n_estimators: 100
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.63773
mae,2859.33892
mse,25294107.16763
r2,0.63775
rmse,5029.32472


wandb: Agent Starting Run: j7gufnv3 with config:
wandb: 	boosting_type: gbdt
wandb: 	learning_rate: 0.01
wandb: 	metric: rmse
wandb: 	n_estimators: 200
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86014
mae,1399.74609
mse,9765561.69688
r2,0.86014
rmse,3124.98987


wandb: Agent Starting Run: csfheg94 with config:
wandb: 	boosting_type: rf
wandb: 	learning_rate: 0.07
wandb: 	metric: rmse
wandb: 	n_estimators: 300
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86433
mae,1367.72399
mse,9472999.41373
r2,0.86433
rmse,3077.82381


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## LGBM Regressor Results

<img src="./LGBM Regressor.png" alt="LGBM Regressor Results"/>